In [2]:
#!pip install ipywidgets==8.1.5

!pip install ipywidgets --upgrade
!pip install jupyterlab_widgets --upgrade

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tensorflow.keras.models import load_model, Model
from tensorflow.keras import layers, models, optimizers, regularizers, initializers
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import HeNormal, HeUniform

In [4]:
feature_extraction_model = load_model('/kaggle/input/fine-tuned-feature-extractor/fine_tuned_best_model.keras')
feature_extraction_model.summary()

feature_extraction_model = Model(inputs=feature_extraction_model.input, outputs = feature_extraction_model.get_layer('global_max_pooling2d').output)
for layer in feature_extraction_model.layers:
    layer.trainable = False
    
feature_extraction_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 27,861,829 (106.28 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 27,861,827 (106.28 MB)

...

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

In [5]:
# Function to extract features from a list of image paths
def extract_features_from_paths(image_paths, model):
    features = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        feature = model.predict(img_array)
        features.append(feature.flatten())
    return np.array(features)

# Function to extract features from a single image
def extract_features_from_image(image_path, model):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    feature = model.predict(img_array)
    return feature.flatten()


# Function to print class information
def print_class_info(path):
    classes = os.listdir(path)
    print("Found classes:")
    for class_name in classes:
        class_path = os.path.join(path, class_name)
        num_images = len(os.listdir(class_path))
        print(f"  - {class_name}: {num_images} images")

In [6]:
import numpy as np
import pickle

# Load the selected features
X_train = np.load('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/X_train_selected.npy')  # Training features
X_val = np.load('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/X_val_selected.npy')      # Validation features
X_test = np.load('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/X_test_selected.npy')    # Test features

y_train = np.load('//kaggle/input/fined-tuned-no-gpu-vgg16-boruta/y_train.npy')    # Training labels
y_val = np.load('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/y_val.npy')        # Validation labels
y_test = np.load('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/y_test.npy')      # Test labels

# Print shapes to verify
print("Validation features shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)
print("Training features shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Test features shape:", X_test.shape)
print("Test labels shape:", y_test.shape)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print(X_train)
print(X_val)
print(X_test)
with open("/kaggle/working/scaler.pkl", 'wb') as file:
    pickle.dump(scaler, file)


Validation features shape: (760, 282)
Validation labels shape: (760,)
Training features shape: (5329, 282)
Training labels shape: (5329,)
Test features shape: (384, 282)
Test labels shape: (384,)
[[-0.57542086 -0.6425264   1.619605   ... -0.01883043  0.44061062
  -0.38586247]
 [ 0.04552769 -0.6425264   0.12169423 ... -0.45196626 -0.1928281
  -0.38586247]
 [ 0.27075988 -0.6425264   1.431495   ... -0.24597101 -0.17930107
  -0.38586247]
 ...
 [ 2.3132696  -0.550432   -0.4143676  ...  1.0260643  -0.585136
  -0.38586247]
 [ 0.8140047   2.2035651  -0.82002294 ... -0.93503153 -0.585136
  -0.38586247]
 [-0.2337771  -0.6425264   2.7919343  ... -0.93503153  0.3863508
  -0.38586247]]
[[-3.5813937e-04 -6.4252639e-01  5.1683497e-01 ... -1.4015448e-01
   7.2808191e-02 -3.8586247e-01]
 [-6.8501514e-01 -1.3429451e-01  1.7320472e+00 ... -4.6469069e-01
   6.7434438e-02  7.1421969e-01]
 [ 2.1451755e-01  5.0417387e-01  5.7606161e-02 ...  2.4234607e+00
  -6.6352688e-02  7.5783783e-01]
 ...
 [-6.0640830e-01

In [7]:
import pickle
# Replace 'path_to_model.pkl' with the path to your pickle file
with open('/kaggle/input/fined-tuned-no-gpu-vgg16-boruta/boruta_selector.pkl', 'rb') as file:
    boruta_selector = pickle.load(file)
selected_feature_indices = boruta_selector.support_



In [8]:
from tensorflow.keras.models import load_model

# Load your trained model
model_path = '/kaggle/input/metrics-modified-with-boruta/best_model_accuracy.keras'
classification_model = load_model(model_path)

# Check the model summary to verify it's loaded correctly
classification_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 282)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2048)           │       579,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 864)            │     1,770,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 864)            │         3,456 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 864)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │         2,595 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,080,843 (27.01 MB)

 Trainable params: 2,358,339 (9.00 MB)

 Non-trainable params: 5,824 (22.75 KB)

 Optimizer params: 4,716,680 (17.99 MB)

In [16]:
import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io

class_names = ['premature', 'mature', 'overmature']

def predict_single_image(single_image_path):
    # Extract features for the single image
    single_image_features = extract_features_from_image(single_image_path, feature_extraction_model)
    
    # Reshape and filter features based on selected indices
    single_image_features = single_image_features.reshape(1, -1)
    single_image_features = single_image_features[:, selected_feature_indices]
    
    # Perform any necessary scaling
    single_image_features = scaler.transform(single_image_features)
    
    # Predict using the classification model
    predicted_probabilities = classification_model.predict(single_image_features)
    predicted_class_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class, predicted_probabilities[0]


image_upload = widgets.FileUpload(accept='image/*', multiple=True)

# Output for image and buttons
image_output = widgets.Output()

# Output for predictions
prediction_output = widgets.Output()

def on_image_upload(change):
    image_output.clear_output()  # Clear previous image and buttons
    prediction_output.clear_output()  # Clear previous predictions
    
    for uploaded_file in image_upload.value:
        with image_output:
            # Access the uploaded file content
            image_data = uploaded_file.content
            image = Image.open(io.BytesIO(image_data))
            
            # Display the image
            plt.imshow(image)
            plt.axis('off')
            plt.show()
            
            # Create a predict button for each uploaded image
            predict_button = widgets.Button(description=f"Predict {uploaded_file.name}")
            
            # Define the callback function for the button
            def on_predict_button_click(b, uploaded_file=uploaded_file):
                with prediction_output:
                    prediction_output.clear_output()  # Clear previous predictions
                    
                    print(f"Button clicked for file: {uploaded_file.name}")
                    # Access the uploaded file content and make prediction
                    image_data = uploaded_file.content
                    image = Image.open(io.BytesIO(image_data))
                    
                    # Save the image to the Kaggle working directory
                    temp_image_path = '/kaggle/working/' + uploaded_file.name
                    with open(temp_image_path, 'wb') as f:
                        f.write(image_data)
                    
                    # Predict the class and probabilities
                    predicted_class, predicted_probabilities = predict_single_image(temp_image_path)
                    
                    # Display predicted class and confidence levels
                    print(f"Predicted class: {predicted_class}")
                    print("Confidence levels:")
                    for i, class_name in enumerate(class_names):
                        print(f"{class_name}: {predicted_probabilities[i]:.4f}")

            # Link the button to the prediction function
            predict_button.on_click(on_predict_button_click)
            
            # Display the predict button
            display(predict_button)

# Link the image upload widget to the function
image_upload.observe(on_image_upload, names='value')

# Display the widgets
display(widgets.VBox([image_upload, image_output, prediction_output]))


In [10]:
# Test a simple button outside the image upload logic to see if it responds to clicks.
test_button = widgets.Button(description="Test Button")

def test_button_callback(b):
    print("Test Button Clicked!")

test_button.on_click(test_button_callback)
display(test_button)

Button(description='Test Button', style=ButtonStyle())

import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io

class_names = ['premature', 'mature', 'overmature']

def predict_single_image(single_image_path):
    # Extract features for the single image
    single_image_features = extract_features_from_image(single_image_path, feature_extraction_model)
    
    # Reshape and filter features based on selected indices
    single_image_features = single_image_features.reshape(1, -1)
    single_image_features = single_image_features[:, selected_feature_indices]
    
    # Perform any necessary scaling
    single_image_features = scaler.transform(single_image_features)
    
    # Predict using the classification model
    predicted_probabilities = classification_model.predict(single_image_features)
    predicted_class_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class, predicted_probabilities[0]


image_upload = widgets.FileUpload(accept='image/*', multiple=False)


output = widgets.Output()

def on_image_upload(change):
    output.clear_output()  # Clear previous outputs
    for uploaded_file in image_upload.value:
        with output:
            # Load the uploaded image using io.BytesIO
            image_data = image_upload.value[uploaded_file]['content']
            image = Image.open(io.BytesIO(image_data))
            
            # Save the image to a temporary path (modify this part based on your environment)
            temp_image_path = '/tmp/' + uploaded_file
            with open(temp_image_path, 'wb') as f:
                f.write(image_data)
            
            # Predict the class and probabilities
            predicted_class, predicted_probabilities = predict_single_image(temp_image_path)
            
            # Display the image
            plt.imshow(image)
            plt.axis('off')
            plt.show()
            
            # Display predicted class
            print(f"Predicted class: {predicted_class}")
            
            # Display confidence levels for each class
            print("Confidence levels:")
            for i, class_name in enumerate(class_names):
                print(f"{class_name}: {predicted_probabilities[i]:.4f}")

# Link the image upload widget to the function
image_upload.observe(on_image_upload, names='value')

# Display the widget
display(widgets.VBox([image_upload, output]))

import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io

class_names = ['premature', 'mature', 'overmature']

def predict_single_image(single_image_path):
    # Extract features for the single image
    single_image_features = extract_features_from_image(single_image_path, feature_extraction_model)
    
    # Reshape and filter features based on selected indices
    single_image_features = single_image_features.reshape(1, -1)
    single_image_features = single_image_features[:, selected_feature_indices]
    
    # Perform any necessary scaling
    single_image_features = scaler.transform(single_image_features)
    
    # Predict using the classification model
    predicted_probabilities = classification_model.predict(single_image_features)
    predicted_class_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class, predicted_probabilities[0]


image_upload = widgets.FileUpload(accept='image/*', multiple=True)

output_list = widgets.VBox([])  # This will hold the list of images with their "Predict" buttons
results_output = widgets.Output()  # This will display the predictions


def on_image_upload(change):
    output_list.children = []  # Clear previous list
    results_output.clear_output()  # Clear previous prediction results
    
    for uploaded_file in image_upload.value:
        # Create a temporary display for each uploaded image
        image_data = image_upload.value[uploaded_file]['content']
        image = Image.open(io.BytesIO(image_data))
        
        # Save the image to a temporary path (modify this part based on your environment)
        temp_image_path = '/tmp/' + uploaded_file
        with open(temp_image_path, 'wb') as f:
            f.write(image_data)

        # Create an image display widget
        img_widget = widgets.Image(
            value=image_data,
            format='png',
            width=150
        )
        
        # Create a button for predicting this image
        predict_button = widgets.Button(description='Predict', button_style='info')
        
        # Function to predict when the button is clicked
        def on_predict_button_click(b, image_path=temp_image_path, image_display=image):
            with results_output:
                results_output.clear_output()  # Clear previous results
                plt.imshow(image_display)
                plt.axis('off')
                plt.show()

                # Predict the class and probabilities
                predicted_class, predicted_probabilities = predict_single_image(image_path)
                
                # Display predicted class
                print(f"Predicted class: {predicted_class}")
                
                # Display confidence levels for each class
                print("Confidence levels:")
                for i, class_name in enumerate(class_names):
                    print(f"{class_name}: {predicted_probabilities[i]:.4f}")

        predict_button.on_click(on_predict_button_click)
        
        # Add the image and button to the output list
        output_list.children = list(output_list.children) + [widgets.HBox([img_widget, predict_button])]


# Link the image upload widget to the function
image_upload.observe(on_image_upload, names='value')

# Display the widget
display(widgets.VBox([image_upload, output_list, results_output]))


import ipywidgets as widgets
from IPython.display import display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import io

class_names = ['premature', 'mature', 'overmature']

def predict_single_image(single_image_path):
    # Extract features for the single image
    single_image_features = extract_features_from_image(single_image_path, feature_extraction_model)
    
    # Reshape and filter features based on selected indices
    single_image_features = single_image_features.reshape(1, -1)
    single_image_features = single_image_features[:, selected_feature_indices]
    
    # Perform any necessary scaling
    single_image_features = scaler.transform(single_image_features)
    
    # Predict using the classification model
    predicted_probabilities = classification_model.predict(single_image_features)
    predicted_class_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class, predicted_probabilities[0]


image_upload = widgets.FileUpload(accept='image/*', multiple=False)


output = widgets.Output()

def on_image_upload(change):
    output.clear_output()  # Clear previous outputs
    for uploaded_file in image_upload.value:
        with output:
            # Access the uploaded file content
            image_data = uploaded_file.content
            image = Image.open(io.BytesIO(image_data))
            
            # Save the image to a temporary path (modify this part based on your environment)
            temp_image_path = '/tmp/' + uploaded_file.name
            with open(temp_image_path, 'wb') as f:
                f.write(image_data)
            
            # Predict the class and probabilities
            predicted_class, predicted_probabilities = predict_single_image(temp_image_path)
            
            # Display the image
            plt.imshow(image)
            plt.axis('off')
            plt.show()
            
            # Display predicted class
            print(f"Predicted class: {predicted_class}")
            
            # Display confidence levels for each class
            print("Confidence levels:")
            for i, class_name in enumerate(class_names):
                print(f"{class_name}: {predicted_probabilities[i]:.4f}")

# Link the image upload widget to the function
image_upload.observe(on_image_upload, names='value')

# Display the widget
display(widgets.VBox([image_upload, output]))


import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import io
import os

class_names = ['premature', 'mature', 'overmature']

# Assuming 'classification_model', 'feature_extraction_model', 'scaler', and 'selected_feature_indices' are already defined.

def predict_single_image(single_image_path):
    # Extract features for the single image
    single_image_features = extract_features_from_image(single_image_path, feature_extraction_model)
    
    # Reshape and filter features based on selected indices
    single_image_features = single_image_features.reshape(1, -1)
    single_image_features = single_image_features[:, selected_feature_indices]
    
    # Perform any necessary scaling
    single_image_features = scaler.transform(single_image_features)
    
    # Predict using the classification model
    predicted_probabilities = classification_model.predict(single_image_features)
    predicted_class_index = np.argmax(predicted_probabilities, axis=1)[0]
    predicted_class = class_names[predicted_class_index]
    
    return predicted_class, predicted_probabilities[0]


def display_image(image_data):
    # Convert image data to an Image object
    image = Image.open(io.BytesIO(image_data))
    
    # Display the image using matplotlib
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
    return image


def on_image_upload(image_data, filename):
    # Save the uploaded image to a temporary path (modify as per your environment)
    temp_image_path = f'/tmp/{filename}'
    
    with open(temp_image_path, 'wb') as f:
        f.write(image_data)
    
    # Display the image
    image = display_image(image_data)

    # Predict the class and probabilities
    predicted_class, predicted_probabilities = predict_single_image(temp_image_path)
    
    # Display predicted class and confidence levels
    print(f"Predicted class: {predicted_class}")
    print("Confidence levels:")
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: {predicted_probabilities[i]:.4f}")


# Assuming you will upload files through the Kaggle notebook interface
# You can use the following code to test it out with a direct image path:

image_path = '/kaggle/input/transparency-insured/New folder (2)/test/1/1 (1198).png'  # Replace with actual image path
with open(image_path, 'rb') as file:
    image_data = file.read()

filename = os.path.basename(image_path)
on_image_upload(image_data, filename)
